In [12]:
import googlemaps
from geo import get_distance_from_home_in_km
from config import settings
import pandas as pd
import arrow


In [13]:
from google.cloud import bigquery
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    filename=settings.GOOGLE_CREDENTIALS_PATH,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [14]:
gmaps = googlemaps.Client(key=settings.GOOGLE_MAPS_API_KEY)

def get_geocode_by_place_name(place_name, visited) -> dict:
    geocode_result = gmaps.geocode(place_name)
    if len(geocode_result) == 0:
        print(f'No results found for {place_name}')
        return {}
    datetime = arrow.now().format('YYYY-MM-DD HH:mm:ss')
    latitude = geocode_result[0]['geometry']['location']['lat']
    longitude = geocode_result[0]['geometry']['location']['lng']
    gdict = {
        'place_name_list': place_name,
        'formatted_address': geocode_result[0]['formatted_address'],
        'location': f'{latitude},{longitude}',
        'place_id': geocode_result[0]['place_id'],
        'visited': visited,
        'distance_from_home_in_km': get_distance_from_home_in_km((latitude, longitude)),
        'datetime': datetime
    }
    return gdict

In [15]:
df = pd.read_csv('data/places.csv')
df.drop_duplicates(inplace=True)
df['visited'] = df['visited'].astype(bool)
df

,places,visited
0,Uxmal,False
1,Progreso,False
2,Telchac,False
3,Celestún,False
4,Cenote maní Chan,False
...,...,...
125,La pirámide de Cholula,True
126,Jardines de México morelos,True
127,San Miguel de Allende,True
128,Hierve el Agua,False


In [16]:
## Geocoding
final_list = [get_geocode_by_place_name(place, visited) for place, visited in zip(df['places'], df['visited'])]
places_df = pd.DataFrame(final_list)
places_df.head()


,place_name_list,formatted_address,location,place_id,visited,distance_from_home_in_km,datetime
0,Uxmal,"Uxmal, Yucatan, Mexico","20.3599062,-89.7683833",ChIJTzynvoY0Vo8RmghR7mqrbVk,False,1052.381902,2023-06-23 11:50:01
1,Progreso,"Progreso, Yucatan, Mexico","21.2811908,-89.66516279999999",ChIJ0yPvvd3CVY8RLV21eHL38_w,False,1071.723086,2023-06-23 11:50:01
2,Telchac,"97407 Telchac Puerto, Yucatan, Mexico","21.3419002,-89.2636045",ChIJ4QxZwIswVI8RThFSar9fFJQ,False,1113.858931,2023-06-23 11:50:01
3,Celestún,"97367 Celestún, Yucatan, Mexico","20.8593042,-90.3972052",ChIJk-uEG7TF-IUR1zjkKOJOb1Q,False,990.835485,2023-06-23 11:50:01
4,Cenote maní Chan,"Entre homun y cuzama, carretera haciendayaxkuk...","20.7117615,-89.30944579999999",ChIJgQlRcfSNVo8R8PDWt1mFTxw,False,1102.190623,2023-06-23 11:50:01


In [17]:
places_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   place_name_list           129 non-null    object 
 1   formatted_address         129 non-null    object 
 2   location                  129 non-null    object 
 3   place_id                  129 non-null    object 
 4   visited                   129 non-null    bool   
 5   distance_from_home_in_km  129 non-null    float64
 6   datetime                  129 non-null    object 
dtypes: bool(1), float64(1), object(5)
memory usage: 6.3+ KB


In [18]:
places_df.drop_duplicates(inplace=True)
places_df['place_name_list'].to_string(index=False)
places_df['formatted_address'].to_string(index=False)
places_df['place_id'].to_string(index=False)


'ChIJTzynvoY0Vo8RmghR7mqrbVk\nChIJ0yPvvd3CVY8RLV21eHL38_w\nChIJ4QxZwIswVI8RThFSar9fFJQ\nChIJk-uEG7TF-IUR1zjkKOJOb1Q\nChIJgQlRcfSNVo8R8PDWt1mFTxw\nChIJsROHu8gKUY8Rs6aA50vFMU4\nChIJxaJZDBWNVo8Rj8pGj7v5Yv4\nChIJxQWNwwpFUo8RFccJDy_Ipu8\nChIJPcsrG4GkVo8RNgEZfUMVVVI\nChIJl11soik_UY8RSO0deqb8H8w\nChIJvV16eGdyUo8RLI8jM6tyhqw\nChIJ8eKrsHeWTY8RTuV3zE_cbqU\nChIJxe8nVwkH8YURxZF2aTCRX50\nChIJ_YV3GAwt94URbQfezmQXMqI\nChIJS2nHF8pV_4URu36MOrR2Hww\nChIJ45FElP7IwIUR_Y671emlmy0\nChIJsdkm30kix4URO_zfV3aniKw\nChIJ4QEGPEAouYUReCgahau-kfY\nChIJZflwMPf3uIURUCu1qhrqLI0\nChIJ87Sj2cMpuYURxi6VXo0YXYE\nChIJP7nTd-n3uIURpKCTb4nhhJE\nChIJA6FQSPYgYYYR9Luq_R1xJzk\nChIJ62bJA8Y69o4RfXwr8Ciq__8\nChIJL4hL4jYouYURrSukh2DVYQg\nChIJcdtDuz0ouYURbuUmlt9IYGk\nChIJo5afrBMouYURz3IgSD352mo\nChIJd9Mrw0DAv4URaNZJtJ9vDvA\nChIJtW8iR33Av4URZfMASSogqnI\nChIJn7SetWXYuIURVmk5ei5yo_w\nChIJs_Bxe1uxuIUR7V6K-S7ZGbI\nChIJt2eSl54lTI8Rxftq1eURSBg\nChIJO3q8Xr3Az4URla2U5B1Gpkg\nChIJ0U7kxypQK4QR4_Wealz5yho\nChIJzZYbezpp1IURuhdi_gy02Cg\nChIJW18x9HxOg

In [19]:
places_df.head()

,place_name_list,formatted_address,location,place_id,visited,distance_from_home_in_km,datetime
0,Uxmal,"Uxmal, Yucatan, Mexico","20.3599062,-89.7683833",ChIJTzynvoY0Vo8RmghR7mqrbVk,False,1052.381902,2023-06-23 11:50:01
1,Progreso,"Progreso, Yucatan, Mexico","21.2811908,-89.66516279999999",ChIJ0yPvvd3CVY8RLV21eHL38_w,False,1071.723086,2023-06-23 11:50:01
2,Telchac,"97407 Telchac Puerto, Yucatan, Mexico","21.3419002,-89.2636045",ChIJ4QxZwIswVI8RThFSar9fFJQ,False,1113.858931,2023-06-23 11:50:01
3,Celestún,"97367 Celestún, Yucatan, Mexico","20.8593042,-90.3972052",ChIJk-uEG7TF-IUR1zjkKOJOb1Q,False,990.835485,2023-06-23 11:50:01
4,Cenote maní Chan,"Entre homun y cuzama, carretera haciendayaxkuk...","20.7117615,-89.30944579999999",ChIJgQlRcfSNVo8R8PDWt1mFTxw,False,1102.190623,2023-06-23 11:50:01


In [20]:
places_df.head()

,place_name_list,formatted_address,location,place_id,visited,distance_from_home_in_km,datetime
0,Uxmal,"Uxmal, Yucatan, Mexico","20.3599062,-89.7683833",ChIJTzynvoY0Vo8RmghR7mqrbVk,False,1052.381902,2023-06-23 11:50:01
1,Progreso,"Progreso, Yucatan, Mexico","21.2811908,-89.66516279999999",ChIJ0yPvvd3CVY8RLV21eHL38_w,False,1071.723086,2023-06-23 11:50:01
2,Telchac,"97407 Telchac Puerto, Yucatan, Mexico","21.3419002,-89.2636045",ChIJ4QxZwIswVI8RThFSar9fFJQ,False,1113.858931,2023-06-23 11:50:01
3,Celestún,"97367 Celestún, Yucatan, Mexico","20.8593042,-90.3972052",ChIJk-uEG7TF-IUR1zjkKOJOb1Q,False,990.835485,2023-06-23 11:50:01
4,Cenote maní Chan,"Entre homun y cuzama, carretera haciendayaxkuk...","20.7117615,-89.30944579999999",ChIJgQlRcfSNVo8R8PDWt1mFTxw,False,1102.190623,2023-06-23 11:50:01


In [21]:
table_id = f'{settings.BQ_DATASET_PREFIX}.places'
bq_client = bigquery.Client(credentials=credentials, project=credentials.project_id)
bq_client.delete_table(table_id, not_found_ok=True)
job = bq_client.load_table_from_dataframe(
    places_df, 
    table_id
)
job.result()

LoadJob<project=minsau-data-project, location=US, id=da759c86-52b2-4475-8ded-2cd308e1b155>